In [1]:
#@markdown <center><h3><h2>VM Video Downloader</h2>Download Vimeo videos easy v0.2</h3></center><br>

#@markdown ---
#@markdown ####Settings

import os
import sys
import base64
import requests
import subprocess
from google.colab import files

from tqdm import tqdm

class VimeoDownloader:
  def __init__(self, url, filename):
    if 'master.json' in url:
      url = url[:url.find('?')] + '?query_string_ranges=1'
      url = url.replace('master.json', 'master.mpd')
    self.url = url
    if not filename.endswith('.mp4'):
      filename += '.mp4'
    self.filename = filename
  def save(self):
    try:
      files.download(self.filename)
      print("If download does not start, download the file manually from the directory in the left corner.")
    except FileNotFoundError:
      print("File not found!")
    except:
      print("Something else went wrong, try download manually")
  def fetch(self, what, to, base):
    with open(to, 'wb') as file:
      init_segment = base64.b64decode(what['init_segment'])
      file.write(init_segment)
      for segment in tqdm(what['segments']):
        segment_url = base + segment['url']
        resp = requests.get(segment_url, stream=True)
        if resp.status_code != 200:
          print('Segment not found:', segment_url)
          break
        for chunk in resp:
            file.write(chunk)
  def download(self):
    if 'master.mpd' in self.url:
      subprocess.run(['youtube-dl', self.url, '-o', self.filename])
      return
    base_url = self.url[:self.url.rfind('/', 0, -26) + 1]
    content = requests.get(self.url).json()
    vid_heights = [(i, d['height']) for (i, d) in enumerate(content['video'])]
    vid_idx, _ = max(vid_heights, key=lambda _h: _h[1])
    audio_quality = [(i, d['bitrate']) for (i, d) in enumerate(content['audio'])]
    audio_idx, _ = max(audio_quality, key=lambda _h: _h[1])
    video = content['video'][vid_idx]
    audio = content['audio'][audio_idx]
    base_url = base_url + content['base_url']
    video_tmp_file = 'video.mp4'
    audio_tmp_file = 'audio.mp4'
    self.fetch(video, video_tmp_file, base_url + video['base_url'])
    self.fetch(audio, audio_tmp_file, base_url + audio['base_url'])
    command = ["ffmpeg", "-i", audio_tmp_file, "-i", video_tmp_file, "-c", "copy", self.filename]
    try:
      result = subprocess.run(command, capture_output=True, text=True, check=True)
      print(result.stdout)  # Exibe a saída do comando
    except subprocess.CalledProcessError as e:
      print(f"Error: {e}")
      print(f"Output error: {e.stderr}")
    os.remove(video_tmp_file)
    os.remove(audio_tmp_file)



url = "https://vod-adaptive-ak.vimeocdn.com/exp=1727036534~acl=%2F692f58e4-b5dc-4f9b-8dcb-866da519ceda%2F%2A~hmac=b7f5b7fd14e71031406a33d415716896490c75af8fd1ba1b455b4873543ac14c/692f58e4-b5dc-4f9b-8dcb-866da519ceda/v2/playlist/av/primary/playlist.json?omit=av1-hevc&pathsig=8c953e4f~3uNjPUkou8dv0609dGLIjG1NQRcGyTD0bKeXgNLxEs0&rh=1cWAMv"  # @param {type:"string"}
file_name = "temp" # @param {type:"string"}
download_to_pc = False #@param {type:"boolean"}
#@markdown ---

downloader = VimeoDownloader(url, file_name)
downloader.download()
if download_to_pc:
  downloader.save()

#@markdown ####How to find the playlist.json?
#@markdown #####Right click on the video > Inspect > Click on the Network tab > Play the video > Right click on "playlist.json?omit=..." > Copy the Request URL, something like this: vod-adaptive-ak.vimeocdn.com/exp=...


#@markdown ---
#@markdown #####Inspired by
#@markdown ##### https://gist.github.com/alexeygrigorev/a1bc540925054b71e1a7268e50ad55cd?permalink_comment_id=5097364#gistcomment-5097364

100%|██████████| 57/57 [00:15<00:00,  3.79it/s]


In [ ]:
#@markdown ####Run this to download all downloaded videos
#@markdown #####Use this only if you didn't marked "download_to_pc" option
def download_all_mp4_files():
    files_in_dir = os.listdir('.')
    mp4_files = [f for f in files_in_dir if f.endswith('.mp4')]

    for file_name in mp4_files:
        files.download(file_name)
        print(f"Downloading {file_name}...")

download_all_mp4_files()